In [1]:
#add to analysis notes (currently no internet connection):
# alpha_diversity.py -i table_even5721.biom -m shannon -o table_even5721_shannon_diversity.txt

In [2]:
import pandas as pd
import asd
import numpy as np

sample_md_fp = 'combined-map.tsv'
faithpd_fp = "../../microbiome-data/run1-4/vsearch-100/cd_even5721/arare_max5721/alpha_div_collated/PD_whole_tree.txt"
oo_fp = "../../microbiome-data/run1-4/vsearch-100/cd_even5721/arare_max5721/alpha_div_collated/observed_otus.txt"

sample_md = pd.read_csv(sample_md_fp, sep='\t', index_col=0, dtype=object)
faithpd = pd.read_csv(faithpd_fp, sep='\t')
oo = pd.read_csv(oo_fp, sep='\t')

sample_md = asd.add_alpha_diversity(sample_md, faithpd, 'Faith PD')
sample_md = asd.add_alpha_diversity(sample_md, oo, 'Observed OTUs')

In [26]:
shannon_diversity = pd.read_csv('../../microbiome-data/run1-4/vsearch-100/cd_even5721/table_even5721_shannon_diversity.txt', 
                                sep='\t', index_col=0, dtype=object)
sample_md['Shannon'] = shannon_diversity
sample_md['Shannon'] = pd.to_numeric(sample_md['Shannon'])

In [27]:
sample_md[['Faith PD', 'Observed OTUs', 'Shannon']].to_csv('./alpha-diversity-even5721.csv')

In [28]:
from scipy.stats import spearmanr


In [29]:
alpha_data = sample_md[['Faith PD', 'Observed OTUs', 'Shannon']].dropna()

In [30]:
spearmanr(alpha_data['Faith PD'], alpha_data['Observed OTUs'])

SpearmanrResult(correlation=0.89990712540521589, pvalue=1.6051073236350284e-206)

In [31]:
spearmanr(alpha_data['Faith PD'], alpha_data['Shannon'])

SpearmanrResult(correlation=0.83362296617555676, pvalue=2.7040937493498394e-148)

In [32]:
spearmanr(alpha_data['Observed OTUs'], alpha_data['Shannon'])

SpearmanrResult(correlation=0.95809230752099617, pvalue=4.9941360541954489e-310)

In [39]:
def tabulate_week_medians(df, sample_type, group):
    df = asd.filter_sample_md(df, [('SampleType', sample_type), ('Group', group)])
    results = []
    df['week'] = pd.to_numeric(df['week'], errors='coerce')
    df = df.sort_values(by='week')
    weeks = df['week'].unique()
    results = []
    for i in weeks:
        result = []
        result.append(np.median(df[df['week'] == i]['Faith PD'].dropna()))
        result.append(np.median(df[df['week'] == i]['Observed OTUs'].dropna()))
        result.append(np.median(df[df['week'] == i]['Shannon'].dropna()))
        results.append(result)
    results = pd.DataFrame(results, index=weeks, columns=['Faith PD', 'Observed OTUs', 'Shannon'])
    return results

In [41]:
week_medians = tabulate_week_medians(sample_md, 'stool', 'autism')

In [42]:
week_medians

,Faith PD,Observed OTUs,Shannon
0,47.153514,1295.55,7.215662
3,46.662844,1335.60,7.301306
10,55.173771,1418.85,7.621539
18,57.419133,1457.25,7.874055


In [45]:
for metric in week_medians.columns:
    print(metric, (week_medians[metric][18] - week_medians[metric][0])/week_medians[metric][0])

Faith PD 0.217706328125
Observed OTUs 0.124811855969
Shannon 0.0912449396715
